<a href="https://colab.research.google.com/github/ChrisHarvey21/HABs/blob/main/HABs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#HABs Model
#creating a model to determine HAB severity based on input variables

In [1]:
%%capture
!pip install geopandas
!pip install libpysal

In [2]:
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from numpy import mean
from numpy import std
from numpy import absolute
from libpysal.weights.contiguity import Queen
import libpysal
from statsmodels.api import OLS
sns.set_style('white')

In [16]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
#load in dataset
#import required function:
import urllib.request

url = 'https://raw.githubusercontent.com/ChrisHarvey21/HABs/main/habs.csv?token=GHSAT0AAAAAABZDDLJKQRCB4PKXEBMIPO7YY3AE6EA'

df1 = pd.read_csv(url)


In [41]:
#View Dataset
df1.head()

,Date,Algal Severity,Precip,Precip 1-day before,precip 2-day before,Sum(3day Precip),Max Temp,Mean Temp
0,9/15/2017,3,0.0,NaN,NaN,0.0,80,68
1,9/16/2017,0,NaN,0.0,NaN,0.0,82,68
2,9/17/2017,0,0.0,NaN,0.0,0.0,84,68
3,9/18/2017,0,0.0,0.0,NaN,0.0,86,71
4,9/19/2017,0,0.0,0.0,0.0,0.0,80,70
